# Data Cleaning – Retail Store Inventory Dataset

## Objective
- Remove invalid or inconsistent records
- Standardize formats and data types
- Prepare data for weekly aggregation and forecasting


In [1]:
import pandas as pd


In [2]:
df = pd.read_csv(
    "../data/raw/retail_store_inventory.csv",
    parse_dates=["Date"]
)

df.shape


(73100, 15)

In [6]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("/", "_")
)

df.columns


Index(['date', 'store_id', 'product_id', 'category', 'region',
       'inventory_level', 'units_sold', 'units_ordered', 'demand_forecast',
       'price', 'discount', 'weather_condition', 'holiday_promotion',
       'competitor_pricing', 'seasonality'],
      dtype='object')

## Core Analytical Columns

- date
- store_id
- product_id
- units_sold (TRUE demand signal)
- inventory_level
- price
- discount
- holiday_promotion


## Columns Excluded from Modeling (for now)

- demand_forecast (to avoid data leakage)


In [12]:
df.isnull().sum()


date                  0
store_id              0
product_id            0
category              0
region                0
inventory_level       0
units_sold            0
units_ordered         0
price                 0
discount              0
weather_condition     0
holiday_promotion     0
competitor_pricing    0
seasonality           0
dtype: int64

In [13]:
df = df.dropna(subset=[
    "date",
    "store_id",
    "product_id",
    "units_sold"
])


In [14]:
df["discount"] = df["discount"].fillna(0)
df["holiday_promotion"] = df["holiday_promotion"].fillna(0)

df["price"] = df.groupby("product_id")["price"].transform(
    lambda x: x.fillna(x.median())
)

df["competitor_pricing"] = df.groupby("product_id")["competitor_pricing"].transform(
    lambda x: x.fillna(x.median())
)


In [15]:
df["weather_condition"] = df["weather_condition"].fillna("Unknown")
df["seasonality"] = df["seasonality"].fillna("Regular")


In [16]:
df = df[df["units_sold"] >= 0]
df = df[df["inventory_level"] >= 0]
df = df[df["units_ordered"] >= 0]


In [ ]:
df = df.drop(columns=["demand_forecast"])


> The provided `demand_forecast` column is excluded to prevent data leakage.
> All forecasting will be performed using historical data only.


In [18]:
df.isnull().sum()
df.describe()


,date,inventory_level,units_sold,units_ordered,price,discount,holiday_promotion,competitor_pricing
count,73100,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000,73100.000000
mean,2022-12-31 23:59:59.999999744,274.469877,136.464870,110.004473,55.135108,10.009508,0.497305,55.146077
min,2022-01-01 00:00:00,50.000000,0.000000,20.000000,10.000000,0.000000,0.000000,5.030000
25%,2022-07-02 00:00:00,162.000000,49.000000,65.000000,32.650000,5.000000,0.000000,32.680000
50%,2023-01-01 00:00:00,273.000000,107.000000,110.000000,55.050000,10.000000,0.000000,55.010000
75%,2023-07-03 00:00:00,387.000000,203.000000,155.000000,77.860000,15.000000,1.000000,77.820000
max,2024-01-01 00:00:00,500.000000,499.000000,200.000000,100.000000,20.000000,1.000000,104.940000
std,NaN,129.949514,108.919406,52.277448,26.021945,7.083746,0.499996,26.191408


In [20]:
df.to_csv("../data/processed/cleaned_data.csv", index=False)


## Cleaning Summary

- Removed rows with missing identifiers or demand values
- Ensured non-negative sales, inventory, and orders
- Filled missing prices and competitor pricing using product medians
- Excluded precomputed demand forecast to avoid leakage
- Preserved all business-relevant signals
